In [1]:
import os
import ROOT
from math import pow, sqrt
from ctypes import c_double

Welcome to JupyROOT 6.28/04


#### Calculate normalization factor for fake rate measurement
#### Based on M(ll) distribution

In [16]:
WORKDIR = "/home/choij/workspace/ChargedHiggsAnalysis"
ERA     = "2016preVFP"
CHANNEL = "MeasFakeMu17"
ID      = "loose"

In [17]:
#### sample list
DataStream = ""
if "El" in CHANNEL:
    if "2016" in ERA:  DataStream = "DoubleEG"
    if "2017" in ERA:  DataStream = "SingleElectron"
    if "2018" in ERA:  DataStream = "EGamma"
if "Mu" in CHANNEL:
    DataStream = "DoubleMuon"

In [18]:
W  = ["WJets_MG"]
DY = ["DYJets", "DYJets10to50_MG"]
TT = ["TTLL_powheg"]
VV = ["WW_pythia", "WZ_pythia", "ZZ_pythia"]
ST = ["SingleTop_sch_Lep", "SingleTop_tch_top_Incl", "SingleTop_tch_antitop_Incl",
      "SingleTop_tW_top_NoFullyHad", "SingleTop_tW_antitop_NoFullyHad"]
MCList = W + DY + TT + VV + ST

In [19]:
SYSTs = []
if "MeasFakeEl" in CHANNEL:
    SYSTs.append(("PileupReweight"))
    SYSTs.append(("L1PrefireUp", "L1PrefireDown"))
    SYSTs.append(("ElectronRecoSFUp", "ElectronRecoSFDown"))
    SYSTs.append(("HeavyTagUpUnCorr", "HeavyTagDownUnCorr"))
    SYSTs.append(("LightTagUpUnCorr", "LightTagDownUnCorr"))
    SYSTs.append(("JetResUp", "JetResDown"))
    SYSTs.append(("JetEnUp", "JetEnDown"))
    SYSTs.append(("ElectronResUp", "ElectronResDown"))
    SYSTs.append(("ElectronEnUp", "ElectronEnDown"))
    SYSTs.append(("MuonEnUp", "MuonEnDown"))
if "MeasFakeMu" in CHANNEL:
    SYSTs.append(("PileupReweight"))
    SYSTs.append(("L1PrefireUp", "L1PrefireDown"))
    SYSTs.append(("MuonRecoSFUp", "MuonRecoSFDown"))
    SYSTs.append(("HeavyTagUpUnCorr", "HeavyTagDownUnCorr"))
    SYSTs.append(("LightTagUpUnCorr", "LightTagDownUnCorr"))
    SYSTs.append(("JetResUp", "JetResDown"))
    SYSTs.append(("JetEnUp", "JetEnDown"))
    SYSTs.append(("ElectronResUp", "ElectronResDown"))
    SYSTs.append(("ElectronEnUp", "ElectronEnDown"))
    SYSTs.append(("MuonEnUp", "MuonEnDown"))

In [20]:
# data
file_path = f"{WORKDIR}/data/MeasFakeRateV3/{ERA}/{CHANNEL}__/DATA/MeasFakeRateV3_{DataStream}.root"
assert os.path.exists(file_path)
f = ROOT.TFile.Open(file_path)
data = f.Get(f"ZEnriched/{ID}/Central/pair/mass"); data.SetDirectory(0)
f.Close()

In [21]:
err_data = c_double()
sum_data = data.IntegralAndError(data.FindBin(50), data.FindBin(150), err_data)
err_data = err_data.value

In [22]:
def getSumMC(syst="Central"):
    sum = 0.
    error = 0.
    for sample in MCList:
        file_path = f"{WORKDIR}/data/MeasFakeRateV3/{ERA}/{CHANNEL}__RunSyst__/MeasFakeRateV3_{sample}.root"
        assert os.path.exists(file_path)
        f = ROOT.TFile.Open(file_path)
        try:
            h = f.Get(f"ZEnriched/{ID}/{syst}/pair/mass");   h.SetDirectory(0)
            thiserr = c_double()
            sum += h.IntegralAndError(h.FindBin(50), h.FindBin(150), thiserr)
            error += pow(thiserr.value, 2)
        except:
            print(sample)
            continue
    f.Close()
    return sum, sqrt(error)

In [23]:
### Get central scale
sumMC = {}
errMC = {}
sum, error = getSumMC()
sumMC["Central"] = sum
errMC["Central"] = error

In [24]:
sum_mc, err_mc = sumMC['Central'], errMC["Central"]
print(sum_data, err_data)
print(sum_mc, err_mc)

142357.0 377.30226609444054
14944802.545334505 7292.135259134853


In [25]:
scale = sum_data / sum_mc
scaleUp = (sum_data + err_data) / (sum_mc - err_mc)
scaleDown = (sum_data - err_data) / (sum_mc + err_mc)
print(scale, scaleUp, scaleDown)
print(max(scaleUp-scale, scale-scaleDown)/scale)

0.009525518960064231 0.009555427802066665 0.009495639291141018
0.003139864833383497


In [26]:
### Get central scale
sumMC = {}
errMC = {}
sum, error = getSumMC()
sumMC["Central"] = sum
errMC["Central"] = error
for systset in SYSTs:
    if len(systset) == 2:
        systUp, systDown = systset
        sumMC[systUp], errMC[systUp] = getSumMC(systUp)
        sumMC[systDown], errMC[systDown] = getSumMC(systDown)
    else:
        syst = systset
        sumMC[syst], errMC[syst] = getSumMC(syst)

In [27]:
# get scale values
scaleMC = {}
for syst, sum in sumMC.items():
    scaleMC[syst] = sum_data / sum
    
diffMC = {}
for syst, sum in scaleMC.items():
    diffMC[syst] = (scaleMC[syst] - scaleMC["Central"]) / scaleMC["Central"]

In [28]:
totalUnc = 0.
for systset in SYSTs:
    if len(systset) == 2:
        systUp, systDown = systset
        diffUp = abs(scaleMC[systUp] - scaleMC["Central"])
        diffDown = abs(scaleMC[systDown] - scaleMC["Central"])
        totalUnc += pow(max(diffUp, diffDown), 2)
    else:
        totalUnc += pow(scaleMC[systset]-scaleMC["Central"], 2)
totalUnc = sqrt(totalUnc)

In [29]:
scale = scaleMC["Central"]
print(scale, totalUnc)
print(totalUnc/scale * 100)

0.009525518960064231 0.00012096592731433805
1.2699142988585512
